In [1]:
import nest_asyncio
nest_asyncio.apply()
from pathlib import Path

import sys
sys.path.append("../")
from typing import Optional, Union, List
import re
import requests
from datetime import datetime

from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)
from desci_sense.shared_functions.configs import MetadataExtractionType
from desci_sense.shared_functions.web_extractors.metadata_extractors import extract_all_metadata_to_dict
from desci_sense.shared_functions.preprocessing import convert_thread_interface_to_ref_post
from desci_sense.shared_functions.dataloaders.twitter.twitter_utils import convert_vxtweet_to_quote_ref_post, convert_vxtweet_to_ref_post
from desci_sense.shared_functions.schema.post import QuoteRefPost, ThreadRefPost, RefPost
from desci_sense.shared_functions.interface import ThreadInterface
from desci_sense.shared_functions.utils import remove_dups_ordered, find_last_occurence_of_any, extract_and_expand_urls, extract_external_urls_from_status_tweet
from desci_sense.shared_functions.prompting.post_renderers.quote_ref_post_renderer import render_quote_post_content
from desci_sense.shared_functions.prompting.post_renderers.thread_ref_post_renderer import ThreadRefPostRenderer

In [2]:
thread_urls = ["https://x.com/EikoFried/status/1798166869574398271",
               "https://x.com/EikoFried/status/1798167612175913332",
               "https://x.com/EikoFried/status/1798170515817013679",
               "https://x.com/EikoFried/status/1798170610314715569",
               "https://x.com/EikoFried/status/1798171316375445681"]

In [3]:
thread_posts = [scrape_post(url) for url in thread_urls]

In [4]:
all_ref_urls = []
for post in thread_posts:
    all_ref_urls += post.md_ref_urls()


In [5]:
def thread_from_posts(posts: List[QuoteRefPost]):
    assert len(posts) > 0
    author = posts[0].author
    content = "\n---\n".join([p.content for p in posts])
    thread_post = ThreadRefPost(author=author,
                  content=content,
                  url=posts[0].url,
                  source_network="twitter",
                  ref_urls=all_ref_urls,
                  posts=thread_posts)
    return thread_post

In [6]:
thread_post = thread_from_posts(thread_posts)

In [7]:
ordered_refs = thread_post.md_ref_urls()

In [8]:
rendered = []
for quote_post in thread_posts:
    rendered.append(render_quote_post_content(quote_post, ordered_refs=ordered_refs))
print("\n---\n".join(rendered))

After careful consideration, the FDA advisory comission voted today 9:2 that MDMA has *not* been shown to be effective for treating PTSD, given massive concerns around validity threats in this literature. They also voted 10:1 that MDMA has *not* shown to be safe. <quoted ref_1>@eturnermd1 #MDMAadcomm VOTE 1/2: Do the available data show that the drug is effective in patients with posttraumatic
stress disorder?
2-Yes
9-No
0-Abstain https://twitter.com/FDAadcomms/status/1798104612635070611/photo/1</quote>
---
📄Many mentioned reasons overlap with those we summarized recently in our review paper: 
<ref_2>

📺 I also summarize them for a lay audience in this YouTube video: 
<ref_3>
---
Some pretty wild things in the meeting honestly, thanks to @eturnermd1 for live tweeting.

Eg folks who were paid by the sponsor (big pharma) to speak on behalf of the product to be marketed did *not* have to declare they were being paid.
---
@eturnermd1 Here is the full thread: <quoted ref_4>Next up on the Ad

In [9]:
thread_post.md_ref_urls()

['https://x.com/FDAadcomms/status/1798104612635070611',
 'https://journals.sagepub.com/doi/10.1177/20451253231198466',
 'https://www.youtube.com/watch?feature=youtu.be&si=kjMtNR1Hwe7NZ8as&v=WknlkmJee4E',
 'https://x.com/eturnermd1/status/1798046087737180395',
 'https://x.com/FDAadcomms/status/1798107142219796794']

In [10]:
md_dict = extract_all_metadata_to_dict(thread_post.md_ref_urls(), md_type=MetadataExtractionType.CITOID, max_summary_length=500)

2024-06-18 18:48:37.776 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-06-18 18:48:37.777 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:104 - skipping citoid for https://x.com/FDAadcomms/status/1798104612635070611
2024-06-18 18:48:37.777 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-06-18 18:48:37.777 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://journals.sagepub.com/doi/10.1177/20451253231198466
2024-06-18 18:48:37.796 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:84 - Retry attempt 1
2024-06-18 18:48:37.797 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:111 - target_url=https://www.youtube.com/watch?feature=youtu.be&si=kjMtNR1Hwe7NZ8as&v=WknlkmJee4E
2024-06-18 18:48:37.797 | WARNING  | 

In [11]:
md_list = list(md_dict.values())

In [12]:
thread_renderer = ThreadRefPostRenderer()

In [14]:
rendered = thread_renderer.render(thread_post, md_list)
print(rendered)


- Author: Eiko Fried
- Content: After careful consideration, the FDA advisory comission voted today 9:2 that MDMA has *not* been shown to be effective for treating PTSD, given massive concerns around validity threats in this literature. They also voted 10:1 that MDMA has *not* shown to be safe. <quoted ref_1>@eturnermd1 #MDMAadcomm VOTE 1/2: Do the available data show that the drug is effective in patients with posttraumatic
stress disorder?
2-Yes
9-No
0-Abstain https://twitter.com/FDAadcomms/status/1798104612635070611/photo/1</quote>
---
📄Many mentioned reasons overlap with those we summarized recently in our review paper: 
<ref_2>

📺 I also summarize them for a lay audience in this YouTube video: 
<ref_3>
---
Some pretty wild things in the meeting honestly, thanks to @eturnermd1 for live tweeting.

Eg folks who were paid by the sponsor (big pharma) to speak on behalf of the product to be marketed did *not* have to declare they were being paid.
---
@eturnermd1 Here is the full thread